In [1]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D

In [2]:
# Model tanımlanır
model=Sequential()
#CNN ve pooling katmanları eklenir
model.add(Conv2D(32,(5,5),activation='relu',input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(32,(5,5),activation='relu'))
model.add(Conv2D(32,(5,5),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#Flatten katmanı eklenir
model.add(Flatten( ))
model.add(Dense(1000,activation='relu') )
model.add(Dense(1,activation='softmax') )# 10 tane etiket olduğu için 10 tane sinir node uluşturuldu.
# Model compile
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

c:\Users\SELCUK\anaconda3\envs\gercek\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
learnedmodel=model.fit(x_train,y_train_ohe,batch_size=256,epochs=2500,validation_split=0.3)